In [1]:
seq1 = "ATCGTAGC"
seq2 = "ATCGTACG"
seq3 = "GCTAGCTA"
seq4 = "TACGATCG"
rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"
protein1 = "ACDEFGHIKLMNPQRSTVWY"
protein2 = "ACDFGHIKLMNPRSTVWY"
protein3 = "ACDEFGHLMNPQRSTVWX"
protein4 = "ABCDEFGHIJKLMN"

1. Алгоритм Нидлмана-Вунша

In [2]:
def Niddleman_Woonsh(seq1, seq2, match_score=1, mismatch_score=-1, gap_score=-2):
    
    rows = len(seq1)+1
    cols = len(seq2)+1
    seq1_list = []
    seq2_list = []

    
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    
    for i in range(1, rows):
        for j in range(1, cols):
            matrix[0][j] =  matrix[0][j-1] + gap_score 
            matrix[i][0] = matrix[i-1][0] + gap_score
            matrix[0][0] = 0

            matrix[i][j] = max((matrix[i-1][j]+gap_score), ((matrix[i-1][j-1] + match_score) if seq1[i-1]==seq2[j-1] else (matrix[i-1][j-1] + mismatch_score)), matrix[i][j-1]+gap_score)
        
    i, j = rows-1, cols-1
    while i > 0 or j > 0:
        current_score = matrix[i][j]
    
        if i > 0 and j > 0 and (current_score == matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)):
            seq1_list += seq1[i-1]
            seq2_list += seq2[j-1]
            i -= 1
            j -= 1
        
        elif i > 0 and current_score == matrix[i-1][j] + gap_score:
            seq1_list += seq1[i-1]
            seq2_list += "-"
            i -= 1
      
        elif j > 0 and current_score == matrix[i][j-1] + gap_score:
            seq1_list += "-"
            seq2_list += seq2[j-1]
            j -= 1

    return ' '.join(map(str, seq1_list[::-1])), ' '.join(map(str, seq2_list[::-1])), matrix, matrix[-1][-1]


In [3]:
Niddleman_Woonsh(rna1, rna2, match_score=1, mismatch_score=-1, gap_score=-2)

('A U G C U A G C -',
 'A U G C U - G C A',
 [[0, -2, -4, -6, -8, -10, -12, -14, -16],
  [-2, 1, -1, -3, -5, -7, -9, -11, -13],
  [-4, -1, 2, 0, -2, -4, -6, -8, -10],
  [-6, -3, 0, 3, 1, -1, -3, -5, -7],
  [-8, -5, -2, 1, 4, 2, 0, -2, -4],
  [-10, -7, -4, -1, 2, 5, 3, 1, -1],
  [-12, -9, -6, -3, 0, 3, 4, 2, 2],
  [-14, -11, -8, -5, -2, 1, 4, 3, 1],
  [-16, -13, -10, -7, -4, -1, 2, 5, 3]],
 3)

In [14]:
Niddleman_Woonsh(seq1, seq2, match_score=1, mismatch_score=-1, gap_score=-2)

('A T C G T A G C',
 'A T C G T A C G',
 [[0, -2, -4, -6, -8, -10, -12, -14, -16],
  [-2, 1, -1, -3, -5, -7, -9, -11, -13],
  [-4, -1, 2, 0, -2, -4, -6, -8, -10],
  [-6, -3, 0, 3, 1, -1, -3, -5, -7],
  [-8, -5, -2, 1, 4, 2, 0, -2, -4],
  [-10, -7, -4, -1, 2, 5, 3, 1, -1],
  [-12, -9, -6, -3, 0, 3, 6, 4, 2],
  [-14, -11, -8, -5, -2, 1, 4, 5, 5],
  [-16, -13, -10, -7, -4, -1, 2, 5, 4]],
 4)

In [5]:
def Smith_Waterman(seq1, seq2, match_score=1, mismatch_score=-1, gap_score=-2):
    rows = len(seq1)+1
    cols = len(seq2)+1
       
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    
    for i in range(1, rows):
        for j in range(1, cols):
            matrix[0][j] =  matrix[0][j-1] + gap_score if (matrix[0][j-1] + gap_score) >= 0 else 0
            matrix[i][0] = matrix[i-1][0] + gap_score if (matrix[i-1][0] + gap_score) >= 0 else 0
            matrix[0][0] = 0
            matrix[i][j] = max((matrix[i-1][j]+gap_score), ((matrix[i-1][j-1] + match_score) if seq1[i-1]==seq2[j-1] else (matrix[i-1][j-1] + mismatch_score)), matrix[i][j-1]+gap_score)
            if matrix[i][j] < 0:
                matrix[i][j] = 0


    
    max_element = max(max(row) for row in matrix)
    max_index = (0, 0)
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == max_element:
                max_index = (i, j)
                break

    n = max_index[0]
    m = max_index[1]

    seq1_list = []
    seq2_list = []

   
    while n > 0 and m > 0 and matrix[n][m] > 0:
        if matrix[n][m] == matrix[n-1][m-1] + (match_score if seq1[n-1] == seq2[m-1] else mismatch_score):
            seq1_list.append(seq1[n-1])
            seq2_list.append(seq2[m-1])
            n -= 1
            m -= 1
        
        elif matrix[n][m] == matrix[n-1][m] + gap_score:
            seq1_list.append(seq1[n-1])
            seq2_list.append('-')
            n -= 1
        
        elif matrix[n][m] == matrix[n][m-1] + gap_score:
            seq1_list.append('-')
            seq2_list.append(seq2[m-1])
            m -= 1

    return ''.join(seq1_list[::-1]), ''.join(seq2_list[::-1]), matrix

In [6]:
Smith_Waterman(seq1='ATCAGAGT', seq2='GTCAGTCT', match_score=3, mismatch_score=-2, gap_score=-2)

('TCAG',
 'TCAG',
 [[0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 3, 1, 0, 0, 0],
  [0, 0, 3, 1, 1, 1, 4, 2, 3],
  [0, 0, 1, 6, 4, 2, 2, 7, 5],
  [0, 0, 0, 4, 9, 7, 5, 5, 5],
  [0, 3, 1, 2, 7, 12, 10, 8, 6],
  [0, 1, 1, 0, 5, 10, 10, 8, 6],
  [0, 3, 1, 0, 3, 8, 8, 8, 6],
  [0, 1, 6, 4, 2, 6, 11, 9, 11]])

3. Реализованные методы в Python.

    1.Библиотека Biopython. Можно использовать модуль Bio.pairwise2. В этом модуле можно выполнять как глобальное, так и локальное варывнивание (align.global--, align.local--). В методы есть две позиции - 1-я отвечает за match и mismatch (можно выставлять x- без штрафов, m - a match score is the score of identical chars, otherwise mismatch score, d - a dictionary returns the score of any pair of characters. c - callback function returns scores.)Такие же параметры есть и для gap(прописываются на втором месте).

In [7]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

alignments = pairwise2.align.globalxx(rna1, rna2)
for alignment in alignments:
    print(format_alignment(*alignment))

AUGCUAGC-
||||| || 
AUGCU-GCA
  Score=7



c:\Users\Anastasia\AppData\Local\Programs\Python\Python311\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [16]:
alignments = pairwise2.align.globalms(seq1, seq2, 1, -1, -2, -0.1)
for alignment in alignments:
    print(format_alignment(*alignment))

ATCGTAGC
||||||..
ATCGTACG
  Score=4



In [19]:
alignments = pairwise2.align.localms('ATCAGAGT', 'GTCAGTCT', 3, -2, -2, -0.1)
for alignment in alignments:
    print(format_alignment(*alignment))

2 TCAGAGT
  ||  |||
2 TC--AGT
  Score=12.9

2 TCAGAGT
  |||  ||
2 TCA--GT
  Score=12.9

2 TCAGAGT
  ||||  |
2 TCAG--T
  Score=12.9



**Вывод: написанный мной алгоритм Ниддлмана-Вунша и алгоритм pairwise2 дают одинаковый результат выравнивания, если выставлять скоринг в pairwaise2, то и score совпадает. Если проверить локальное выравнивание модуля pairwise2, то он выдает больше вариантов, чем написанный мною алгоритм Смита-Ватермана.**

Проверим последовательность из пункта 1 дз:

In [21]:
my_seq1 = 'CGTCTT'
my_seq2 = 'CATTCT'

alignments = pairwise2.align.globalms(my_seq1, my_seq2, 1, -1, -2, -0.0001)
for alignment in alignments:
    print(format_alignment(*alignment))

CGTCTT
|.|..|
CATTCT
  Score=0



**Вывод: Алгоритм Bio.pairwise2 и мое решение совпадают**

2. Parasail

In [67]:
score_size = 1 # 0, use 8-bit align; 1, use 16-bit; 2, try both
profile = parasail.ssw_init("GATTACA", parasail.blosum62, score_size)
result = parasail.ssw_profile(profile, "GCATGCU", 2, 1)
print(result.score1)
print(result.cigar)
print(result.ref_begin1)
print(result.ref_end1)
print(result.read_begin1)
print(result.read_end1)

20
[23 40 23 24 23]
0
5
0
5


3. PairwiseAligner 

In [80]:
from Bio.Align import PairwiseAligner

aligner = PairwiseAligner()
aligner.mode = 'global'
seq1 = "GATTACA"
seq2 = "GCATGCU"
alignments = aligner.align(seq1, seq2)

for alignment in alignments:
    print(alignment)

target            0 G-ATTA-CA-  7
                  0 |-||---|-- 10
query             0 GCAT--GC-U  7

target            0 G-ATTA-CA-  7
                  0 |-|-|--|-- 10
query             0 GCA-T-GC-U  7

target            0 G-ATT-ACA-  7
                  0 |-||---|-- 10
query             0 GCAT-G-C-U  7

target            0 G-ATT-ACA-  7
                  0 |-|-|--|-- 10
query             0 GCA-TG-C-U  7

target            0 G-AT-TACA-  7
                  0 |-||---|-- 10
query             0 GCATG--C-U  7

target            0 G-ATTACA- 7
                  0 |-||.-|-- 9
query             0 GCATG-C-U 7

target            0 G-ATTACA- 7
                  0 |-||-.|-- 9
query             0 GCAT-GC-U 7

target            0 G-ATTACA- 7
                  0 |-|-|.|-- 9
query             0 GCA-TGC-U 7

target            0 G-ATTA-C-A  7
                  0 |-||---|-- 10
query             0 GCAT--GCU-  7

target            0 G-ATTA-C-A  7
                  0 |-|-|--|-- 10
query             0 GCA